<a href="https://colab.research.google.com/github/vasudevgupta7/huggingface-transformers/blob/contribute/translation_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing appropriate modules
!pip install transformers==4.1.1
!pip install git+https://github.com/huggingface/datasets.git@master
!pip install sentencepiece

In [2]:
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, 
    TrainingArguments, Trainer
  )

import torch
from torch.utils.data import random_split

In [ ]:
# lets download the datasets using 🤗
data = load_dataset("mkb", language_pair="en-hi")
data

In [5]:
# splitting dataset into train, validation
split = 0.25
train_dataset, eval_dataset = random_split(data["train"], lengths=[int((1-split)*len(data["train"])), int(split*len(data["train"]))])

In [6]:
def data_collator(features: list):

  labels = [f["translation"]["en"] for f in features]
  inputs = [f["translation"]["hi"] for f in features]

  batch = tokenizer.prepare_seq2seq_batch(src_texts=inputs, src_lang="hi_IN", tgt_lang="en_XX", tgt_texts=labels, max_length=32, max_target_length=32)

  for k in batch:
    batch[k] = torch.tensor(batch[k])

  return batch

In [7]:
# initiating model, tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")

# defining training related arguments
args = TrainingArguments(output_dir="my-mbart",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=16,
                        learning_rate=5e-5,
                        num_train_epochs=2)

Some weights of MBartForConditionalGeneration were not initialized from the model checkpoint at facebook/mbart-large-cc25 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# defining trainer using 🤗
trainer = Trainer(model=model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=train_dataset, 
                eval_dataset=eval_dataset,
                tokenizer=tokenizer)

In [9]:
# lets train model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.068964
2,No log,5.770269


TrainOutput(global_step=496, training_loss=4.525921237084173)